<a href="https://colab.research.google.com/github/rishipatel92/Tensorflow/blob/main/Predicting_College_Placement_Using_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as pt
import tensorflow as tf

In [54]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
df_train = pd.read_excel("/content/drive/MyDrive/Prediction of Placement Status Data/01 Train Data.xlsx")

In [56]:
df_test_m = pd.read_excel("/content/drive/MyDrive/Prediction of Placement Status Data/02 Test Data.xlsx")

In [57]:
df_train = df_train[['CGPA','Speaking Skills','ML Knowledge','Placement Status']]

In [58]:
df_test_c = df_test_m[['CGPA','Speaking Skills','ML Knowledge']]

In [59]:
df_train = df_train.dropna(axis=0)

Replacing the Placement Status with 0 and 1 to train the model

In [60]:
df_train.loc[df_train["Placement Status"] == "Not placed","Placement Status"] = 0
df_train.loc[df_train["Placement Status"] == "Placed","Placement Status"] = 1

In [61]:
x = df_train[['CGPA','Speaking Skills','ML Knowledge']]
y = df_train['Placement Status'].astype('int')

In [62]:
from sklearn.model_selection import train_test_split

In [63]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [64]:
tf.convert_to_tensor(x_train)

<tf.Tensor: shape=(735, 3), dtype=float64, numpy=
array([[7.1, 2. , 3. ],
       [7.4, 4. , 4. ],
       [7.4, 4. , 4. ],
       ...,
       [8.3, 2. , 2. ],
       [7.9, 4. , 4. ],
       [7.3, 2. , 2. ]])>

In [68]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(x_train)

In [70]:
normalizer(x_train.iloc[:3])

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[-0.9672729 , -1.2846429 , -0.44192353],
       [-0.6656413 ,  0.4336693 ,  0.4242462 ],
       [-0.6656413 ,  0.4336693 ,  0.4242462 ]], dtype=float32)>

In [75]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [76]:
SHUFFLE_BUFFER = 500
BATCH_SIZE = 2
model = get_basic_model()
model.fit(x_train, y_train, epochs=15, batch_size=BATCH_SIZE)

Epoch 1/15
368/368 [==============================] - 2s 2ms/step - loss: 0.6670 - accuracy: 0.6694
Epoch 2/15
368/368 [==============================] - 1s 2ms/step - loss: 0.6323 - accuracy: 0.6776
Epoch 3/15
368/368 [==============================] - 1s 2ms/step - loss: 0.6279 - accuracy: 0.6776
Epoch 4/15
368/368 [==============================] - 1s 2ms/step - loss: 0.6272 - accuracy: 0.6776
Epoch 5/15
368/368 [==============================] - 1s 2ms/step - loss: 0.6268 - accuracy: 0.6776
Epoch 6/15
368/368 [==============================] - 1s 2ms/step - loss: 0.6258 - accuracy: 0.6776
Epoch 7/15
368/368 [==============================] - 1s 3ms/step - loss: 0.6254 - accuracy: 0.6776
Epoch 8/15
368/368 [==============================] - 1s 3ms/step - loss: 0.6253 - accuracy: 0.6776
Epoch 9/15
368/368 [==============================] - 1s 4ms/step - loss: 0.6244 - accuracy: 0.6776
Epoch 10/15
368/368 [==============================] - 1s 3ms/step - loss: 0.6226 - accuracy: 0.6776

In [65]:
model = tf.keras.Sequential([
tf.keras.layers.Flatten(input_shape=[735, 3]),
tf.keras.layers.Dense(300, activation="relu"),
tf.keras.layers.Dense(100, activation="relu"),
tf.keras.layers.Dense(10, activation="softmax")
])